In [1]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import os
import glob

def sol_1d_advection(x, q, dt, dx, c, step):
    ''' 輸送速度が正の1次元移流方程式を数値計算する '''
    dir = 'img-Lax'

    # 1次精度風上法との比較用
    q_kaze = q.copy()

    for i in range(step):
        if i % 10 == 0:
            print('Iteration=', i)
        q0 = q.copy()

        # 1次精度風上法との比較用
        q0_kaze = q_kaze.copy()

        for j in range(1, len(x)-1):
            # Lax法
            q[j] = (1 / 2) * (q0[j+1] + q0[j-1]) - (c / 2) * (dt / dx) * (q0[j+1] - q0[j-1])

            # 1次精度風上法（比較用）
            q_kaze[j] = q0_kaze[j] - (1 / 2) * (dt / dx) \
                        * (((c + np.abs(c)) * (q0_kaze[j] - q0_kaze[j-1])) \
                        + ((c - np.abs(c)) * (q0_kaze[j+1] - q0_kaze[j])))

        plot(x, q_kaze, q, i, dir)
    create_gif(dir, '1d_advection_Lax-inverse.gif')
    return


def plot(x, q1, q2, i, dir):
    ''' シミュレーション結果をプロットし画像を保存する '''
    # フォントの種類とサイズを設定する。
    plt.rcParams['font.size'] = 14
    plt.rcParams['font.family'] = 'Times New Roman'

    # 目盛を内側にする。
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'

    # グラフの上下左右に目盛線を付ける。
    fig = plt.figure()

    ax1 = fig.add_subplot(111)
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')

    # 軸のラベルを設定する。
    ax1.set_xlabel('x')
    ax1.set_ylabel('q')

    # スケールの設定をする。
    ax1.set_xlim(np.min(x), np.max(x))
    #ax1.set_xlim(1.5, 3)
    #ax1.set_xlim(7, 8.5)
    ax1.set_ylim(-0.2, 1.2)

    # プロットを行う。
    ax1.plot(x, q1, label='Upwind', lw=1, color='black', linestyle='-')
    ax1.plot(x, q2, label='Lax', lw=1, color='blue')
    ax1.legend(bbox_to_anchor=(0, 1), loc='upper left', borderaxespad=0)

    # レイアウト設定
    fig.tight_layout()

    # 画像を保存する。
    # dirフォルダが無い時に新規作成
    save_dir = dir
    if os.path.exists(save_dir):
        pass
    else:
        os.mkdir(save_dir)

    # 画像保存パスを準備
    path = os.path.join(*[save_dir, str("{:05}".format(i)) + '.png'])

    # 画像を保存する
    plt.savefig(path)

    # グラフを表示する。
    #plt.show()
    plt.close()
    return


def create_gif(in_dir, out_filename):
    ''' imgフォルダの複数画像からGIF画像を作る '''
    path_list = sorted(glob.glob(os.path.join(*[in_dir, '*'])))  # ファイルパスをソートしてリストする
    imgs = []  # 画像をappendするための空配列を定義

    # ファイルのフルパスからファイル名と拡張子を抽出
    for i in range(len(path_list)):
        img = Image.open(path_list[i])  # 画像ファイルを1つずつ開く
        imgs.append(img)  # 画像をappendで配列に格納していく

    # appendした画像配列をGIFにする。durationで持続時間、loopでループ数を指定可能。
    imgs[0].save(out_filename,
                 save_all=True, append_images=imgs[1:], optimize=False, duration=200, loop=0)
    return


if __name__ == '__main__':
    ''' 条件設定を行いシミュレーションを実行、流れのGIF画像を作成する '''
    # 時間項の条件
    dt = 0.05

    # 空間項の条件
    dx = 0.1
    x_min = 0
    x_max = 10
    x = np.arange(x_min, x_max, dx)     # 1次元計算格子

    # 初期分布qと輸送速度c
    q = np.zeros_like(x)
    q[:int(len(x)/5)] = 1
    #c=1
    q = q[::-1]
    c = -1

    # クーラン数の確認
    nu = c * dt / dx
    print('Courant number=', nu)

    # シミュレーションの実行
    sol_1d_advection(x=x, q=q, dt=dt, dx=dx, c=c, step=100)

Courant number= -0.5
Iteration= 0
Iteration= 10
Iteration= 20
Iteration= 30
Iteration= 40
Iteration= 50
Iteration= 60
Iteration= 70
Iteration= 80
Iteration= 90
